In [1]:
import pandas as pd
import numpy as np

In [2]:
expeditions = pd.read_csv('./data/expeditions.csv')
members = pd.read_csv('./data/members.csv')
peaks = pd.read_csv('./data/peaks.csv')

### Question 1

In [42]:
# Get proportion of peaks unclimbed
unclimbed_one = (peaks[peaks['climbing_status'] == 'Unclimbed'].value_counts('climbing_status')/len(peaks))
unclimbed_one # around 27% are still unclimbed


climbing_status
Unclimbed    0.271368
dtype: float64

In [43]:
#OR

unclimbed_two = (peaks['climbing_status'].value_counts()['Unclimbed']/len(peaks))
unclimbed_two

0.27136752136752135

In [33]:
# Get Avg. Height of climbed & unclimbed

avg_height = peaks.groupby('climbing_status')['height_metres'].mean()
avg_height # 6706.3 for Climbed, 6523.3 for Unclimbed

climbing_status
Climbed      6706.284457
Unclimbed    6523.330709
Name: height_metres, dtype: float64

### Question 2

In [66]:
members_citizenship = members[members['citizenship'] == 'Nepal']
members_citizenship # get Sherpas

,expedition_id,member_id,peak_id,peak_name,year,season,sex,age,citizenship,expedition_role,...,highpoint_metres,success,solo,oxygen_used,died,death_cause,death_height_metres,injured,injury_type,injury_height_metres
24,AMAD79101,AMAD79101-18,AMAD,Ama Dablam,1979,Spring,M,23.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
48,AMAD79301,AMAD79301-26,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
49,AMAD79301,AMAD79301-24,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
50,AMAD79301,AMAD79301-25,AMAD,Ama Dablam,1979,Autumn,M,NaN,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
82,AMAD81101,AMAD81101-11,AMAD,Ama Dablam,1981,Spring,M,25.0,Nepal,H-A Worker,...,6814.0,True,False,False,False,NaN,NaN,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76506,PUMO19102,PUMO19102-01,PUMO,Pumori,2019,Spring,M,35.0,Nepal,Leader,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76507,PUMO19102,PUMO19102-02,PUMO,Pumori,2019,Spring,M,52.0,Nepal,H-A Worker,...,7138.0,True,False,False,False,NaN,NaN,False,NaN,NaN
76513,PUTH19101,PUTH19101-06,PUTH,Putha Hiunchuli,2019,Spring,M,NaN,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN
76514,PUTH19101,PUTH19101-07,PUTH,Putha Hiunchuli,2019,Spring,M,44.0,Nepal,H-A Worker,...,6350.0,False,False,False,False,NaN,NaN,False,NaN,NaN


In [56]:
members_cit_hired = members_citizenship['hired'].value_counts()[True]/len(members_citizenship)
members_cit_hired # hired percentage ~ 93.4%

0.9340563991323211

In [69]:
members_cit_age = members_citizenship[['age','citizenship']].groupby('citizenship').agg(['min', 'max', 'mean'])
members_cit_age

age                 
              min   max       mean
citizenship                       
Nepal        12.0  85.0  32.885062